# Задание

Cоздать модель для классификации текста по тональности -  NLP, sentiment analysis.
Классифицировать отзывы на фильмы в IDMB на positive/negative
классы. Датасет доступен по на kaggle по ссылке ( https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews ).




** Цель **  этого задания, произвести эмбеддинг слов для
модели. Сама модель будет простая логистическая регрессия, без
множества слоев, то есть метод классического машинного обучения. Шаги
выполнения задания:
1. Загрузить IMDB датасет
2. Обработать отзывы так, чтобы там были только слова расположенные
через пробел, и при том с маленькой буквы (без знаков препинания).
Для этого можно использовать функционал библиотеки re (regular
expressions). Удалить слова из ENGLISH_STOP_WORDS. Это не цензура, а
просто слова создающие шум и не несущие больше смысла в тексте.
3. Преобразовать текст в его векторное представление с помощью spacy
4. Разделите данные на train/test и натренировать модель логистической
регрессии на подготовленных данных. Для этого нам понадобятся
дополнительные импорты из sklearn.
5. Оценить модель по метрикам accuracy и f1.
6. Выбрать несколько случайных отзывов и классифицировать их, сравнить результат с реальными лэйблами.

In [1]:
!pip install kaggle --quiet
# Создаём папку для ключа
!mkdir -p ~/.kaggle
# Загружаем kaggle.json в Colab вручную
from google.colab import files
files.upload()  # здесь загрузите файл kaggle.json

# Копируем его в нужное место
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # задаём правильные права

Saving kaggle.json to kaggle.json


In [2]:
import kagglehub  # Это библиотека, позволяющая удобно загружать датасеты и модели с платформы Kaggle.
from kagglehub import KaggleDatasetAdapter # Этот компонент используется для подключения и адаптации датасетов Kaggle к Python-программе.

In [3]:
# Загружаем правильный датасет imdb movies: lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
# Распаковываем архив imdb-dataset-of-50k-movie-reviews.zip
!unzip -q imdb-dataset-of-50k-movie-reviews.zip -d imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
  0% 0.00/25.7M [00:00<?, ?B/s]
100% 25.7M/25.7M [00:00<00:00, 1.22GB/s]


In [4]:
!pip install pandas numpy scikit-learn spacy  --quiet
!python -m spacy download en_core_web_md  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 26.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import pandas as pd  # Работа с таблицами (данными)
import numpy as np  # Работа с массивами (векторами)
import re  # Регулярные выражения (обработка текста)
import spacy  # Библиотека для NLP, которая преобразует текст в вектор
from sklearn.model_selection import train_test_split  # Делим данные на train/test
from sklearn.linear_model import LogisticRegression  # Модель логистической регрессии
from sklearn.metrics import accuracy_score, f1_score  # Метрики качества модели
from sklearn.preprocessing import LabelEncoder  # Преобразует текстовые метки (строки) в числа
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS  # Список часто встречающихся английских слов не несущих особого смысла

In [13]:
# Загружаем CSV-файл с отзывами. В колонке "review" — текст, в колонке "sentiment" — метка ('positive' / 'negative')
initial_data = pd.read_csv('/content/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

In [14]:
initial_data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [15]:
initial_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [16]:
initial_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [18]:
# DataFrame очень большой. Поэтому для отладки возьмём первые 100 строк
data = initial_data.head(100)

In [19]:
# Загружаем предобученную модель spaCy. Она содержит векторы слов и умеет обрабатывать текст.
nlp = spacy.load('en_core_web_md')  # Модель со встроенными векторами слов

In [29]:
# Функция для предобработки текста
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # Удаляем знаки препинания и приводим к нижнему регистру
    tokens = text.split()  # Разделяем текст на отдельные слова (токены)
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS]  # Удаляем часто встречающиеся слова (например: the, is, and)
    return ' '.join(tokens)  # Соединяем токены обратно в строку

# Применяем предобработку к каждому отзыву
data['review'] = data['review'].apply(preprocess_text)

/tmp/ipython-input-2544789388.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(preprocess_text)


In [30]:
# Преобразуем текст в вектор при помощи spaCy
def get_spacy_vectors(text):
    doc = nlp(text)  # Обрабатываем текст моделью spaCy
    return doc.vector  # Получаем усреднённый вектор всего текста

# Добавляем новую колонку с векторами
data['spacy_vectors'] = data['review'].apply(get_spacy_vectors)

/tmp/ipython-input-3120564427.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['spacy_vectors'] = data['review'].apply(get_spacy_vectors)


In [31]:
# Подготовка признаков (входных данных X) и целевых переменных (меток y)
X = np.array(data['spacy_vectors'].tolist())  # Преобразуем список векторов в массив NumPy
y = LabelEncoder().fit_transform(data['sentiment'])  # Преобразуем 'positive'/'negative' в 1/0

# Делим данные на обучающую и тестовую выборки (80% и 20% соответственно)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создаём модель логистической регрессии и обучаем её
classifier = LogisticRegression(max_iter=1000)  # max_iter — максимум шагов при обучении
classifier.fit(X_train, y_train)  # Обучение модели

# Делаем предсказания на тестовых данных
y_pred = classifier.predict(X_test)

# Оцениваем модель: точность (accuracy) и F1-мера
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Выводим метрики
print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')




Accuracy: 0.5
F1 Score: 0.5


In [32]:

# Функция для предсказания меток случайных отзывов
def classify_random_reviews(data, classifier, n=5):
    random_reviews = data.sample(n)  # Берём случайные n строк из датафрейма
    for i, row in random_reviews.iterrows():
        text = row['review']  # Исходный текст отзыва
        true_label = row['sentiment']  # Истинная метка
        vector = get_spacy_vectors(text)  # Преобразуем текст в вектор
        predicted_label = classifier.predict([vector])[0]  # Предсказываем класс
        print(f"Review: {text}\nTrue Label: {true_label}\nPredicted Label: {'positive' if predicted_label == 1 else 'negative'}\n")

# Применяем классификацию к случайным отзывам
classify_random_reviews(data, classifier, n=5)

Review: lets beginbr br movie original cronenbergs movies usually appearbr br intention certainly jjl favourite actresses lovely usual cutiebr br say favourite movie quite interesting entertaining follow br br rest cast extremely impressive kind miscast star array br br recommend confidence
True Label: positive
Predicted Label: negative

Review: exposure templarios good excited title offerings anchor bay video brought cult classics spider baby print quality excellent hide fact film deadly dull theres thrilling opening sequence villagers exact terrible revenge templars set thing motion movie slow ponderous ultimately unfulfilling adding insult injury movie dubbed subtitled promised video jacket
True Label: negative
Predicted Label: negative

Review: petter matteis love time money visually stunning film watch mr mattei offers vivid portrait human relations movie telling money power success people different situations encounter br br variation arthur schnitzlers play theme director transf